# NILM Training
This python notebook implements the training part for a NILM model

## Imports

In [1]:
import pandas as pd
import os
from config_loader import load_config
import numpy as np
from joblib import dump
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import MinMaxScaler
import random
import matplotlib.pyplot as plt
import json
from contextlib import redirect_stdout
import plotly.io as pio
import io
import base64

In [2]:
config, config_dir = load_config()

env = config['Settings']['environment']
data_path = config[env]['data_path']
training_dataset_file = config['Data']['training_dataset_file']
model_file = config['Data']['model_file']
models_dir = config['Data']['models_dir']
input_scaler_file = config['Data']['input_scaler_file']
target_scalers_file = config['Data']['target_scalers_file']
scalers_dir = config['Data']['scalers_dir']
max_epochs = int(config['ML']['max_epochs'])
patience = int(config['ML']['patience'])
hidden_size = int(config['ML']['hidden_size'])
sequence_length = int(config['ML']['sequence_length'])
batch_size = int(config['ML']['batch_size'])
column_names_file = config['Data']['training_dataset_columns_file']

SEED = 42

## Selection of the Training Device

In [3]:
# Check for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    random.seed(SEED)
    np.random.seed(SEED)

print("Using device:", device)

# Check CUDA and cuDNN version only if needed
if torch.cuda.is_available():
    print("Torch Version:", torch.__version__)
    print("Cuda Version:", torch.version.cuda)
    print("CUDNN Version:", torch.backends.cudnn.version())
    print("CUDA available:", torch.cuda.is_available())


Using device: cuda
Torch Version: 2.6.0+cu118
Cuda Version: 11.8
CUDNN Version: 90100
CUDA available: True


## Dataset
### Load the dataset

In [4]:
with open(os.path.join(data_path, column_names_file), 'r') as file:
    column_names_json = json.load(file)

timestamp_column = column_names_json['time'][0]
household_id_column = column_names_json['household_info'][0]
smart_meter_columns = column_names_json['sm_metrics']
appliances_columns = column_names_json['appliances']
print(appliances_columns)

path = os.path.join(data_path, training_dataset_file)
dataset = pd.read_parquet(path)
print(dataset.head())

appliance_households = {}

for appliance in appliances_columns:
    valid_rows = dataset[dataset[appliance] != 0]

    # Get unique household_ids for those rows
    valid_households = valid_rows['household_id'].unique().tolist()

    appliance_households[appliance] = valid_households

appliances_dataset = {}

for appliance in appliances_columns:
    # Filter rows for the current appliance
    subset = dataset[dataset['household_id'].isin(appliance_households[appliance])]

    # Select only the current appliance, plus smart meter columns
    cols_to_keep = [timestamp_column] + [household_id_column] + smart_meter_columns + [appliance]
    appliances_dataset[appliance] = subset[cols_to_keep]
    print(appliances_dataset[appliance].head(10))

for appliance in appliances_dataset:
    print(f'Appliance: {appliance}')
    print(appliances_dataset[appliance][appliance].describe())

['Coffee Machine', 'Dryer', 'Freezer', 'Fridge', 'Lamp', 'Laptop', 'Microwave', 'PC', 'Router', 'Tablet', 'Washing Machine', 'Other']
            timestamp household_id  powerl1  powerl2  powerl3  currentl1  \
0 2012-09-16 00:00:00           01  60.0575  18.2049  10.0497    0.45069   
1 2012-09-16 00:00:10           01  60.0084  18.4098   9.9889    0.45268   
2 2012-09-16 00:00:20           01  59.8883  18.3856   9.9568    0.45058   
3 2012-09-16 00:00:30           01  59.9243  18.3527  10.0348    0.45205   
4 2012-09-16 00:00:40           01  59.8486  18.2500   9.9689    0.45277   

   currentl2  currentl3  voltagel1  voltagel2  ...  Freezer    Fridge  Lamp  \
0    0.18567    0.12098    237.301      237.0  ...  37.3928  1.323468   0.0   
1    0.18642    0.12175    237.358      237.0  ...  37.3928  1.323468   0.0   
2    0.18602    0.12096    236.640      236.9  ...  37.6085  0.882312   0.0   
3    0.18611    0.12133    236.143      236.9  ...  37.3928  0.661734   0.0   
4    0.18578  

## Model Training
### Chunking the Dataset

In [5]:
class SlidingWindowDataset(Dataset):
    def __init__(self, data, input_columns, target_columns, sequence_length, stride=1, time_threshold='10s'):
        self.sequence_length = sequence_length
        self.stride = stride
        self.input_columns = input_columns
        self.target_columns = target_columns

        self.data = data.copy()
        self.data['timestamp'] = pd.to_datetime(self.data['timestamp'])

        # Optional: One-hot encode household_id_column if categorical
        if 'household_id_column' in self.input_columns:
            self.data = pd.get_dummies(self.data, columns=['household_id_column'], drop_first=True)

        # Convert input/target columns to numeric
        self.data[self.input_columns] = self.data[self.input_columns].apply(pd.to_numeric, errors='coerce')
        self.data[self.target_columns] = self.data[self.target_columns].apply(pd.to_numeric, errors='coerce')

        # Handle multiple households if column exists
        household_col = 'household_id_column' if 'household_id_column' in data.columns else None
        self.indices = []

        if household_col:
            group_cols = [household_col]
        else:
            group_cols = []

        # Sort by household and timestamp
        self.data = self.data.sort_values(group_cols + ['timestamp']).reset_index(drop=True)

        # Calculate time difference
        if household_col:
            self.data['time_diff'] = self.data.groupby(household_col)['timestamp'].diff()
        else:
            self.data['time_diff'] = self.data['timestamp'].diff()

        # Flag new segments where the time gap exceeds the threshold
        threshold = pd.Timedelta(time_threshold)
        self.data['segment'] = (self.data['time_diff'] > threshold).cumsum()

        # Group by segments (and household if needed)
        groupby_cols = ['segment'] + ([household_col] if household_col else [])

        for _, group in self.data.groupby(groupby_cols):
            group = group.reset_index(drop=True)
            for start in range(0, len(group) - sequence_length + 1, stride):
                self.indices.append((group, start))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        group, start = self.indices[idx]
        x = group[self.input_columns].iloc[start : start + self.sequence_length].values
        y = group[self.target_columns].iloc[start : start + self.sequence_length].values

        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y



### Creating the Model

In [6]:
class NILMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout_rate=0.1):
        super(NILMModel, self).__init__()

        self.hidden_size = hidden_size

        # 1D convolution before LSTM
        self.conv1d = nn.Conv1d(in_channels=input_size, out_channels=input_size,
                                kernel_size=3, padding=1)  # keeps same shape

        # LSTM layer (can be bidirectional)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout_rate)

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Activation
        self.relu = nn.ReLU()

        self.output_layer = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        # x: (batch, seq_len, input_features)
        x = x.permute(0, 2, 1)  # (batch, input_size, seq_len)
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # back to (batch, seq_len, input_size)

        lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden*directions)

        # Dropout
        dropout_out = self.dropout(lstm_out)

        # Activation
        relu_out = self.relu(dropout_out)

        out = self.output_layer(relu_out)  # (batch, seq_len, 1)


        return out


### Dataset Preparation

In [7]:
input_columns = list(smart_meter_columns)

input_scaler = MinMaxScaler()
dataset[input_columns] = input_scaler.fit_transform(dataset[input_columns])
dump(input_scaler, os.path.join(data_path, input_scaler_file))

for appliance in appliances_columns:
    appliances_dataset[appliance][input_columns] = dataset.loc[
        appliances_dataset[appliance].index, input_columns
    ]

target_scalers = {}
for appliance in appliances_columns:
    scaler = MinMaxScaler()
    appliances_dataset[appliance][appliance] = scaler.fit_transform(appliances_dataset[appliance][[appliance]])
    target_scalers[appliance] = scaler

os.makedirs(os.path.join(data_path, scalers_dir), exist_ok=True)

for appliance, scaler in target_scalers.items():
    print(f"{appliance}: min = {scaler.data_min_[0]}, max = {scaler.data_max_[0]}")
    print(f"Target appliance '{appliance}': min = {appliances_dataset[appliance][appliance].min()}, max = {appliances_dataset[appliance][appliance].max()}")
    dump(scaler, os.path.join(data_path, scalers_dir, appliance.lower().replace(' ', '_') + target_scalers_file))

C:\Users\morel\AppData\Local\Temp\ipykernel_15260\1267129062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appliances_dataset[appliance][input_columns] = dataset.loc[
C:\Users\morel\AppData\Local\Temp\ipykernel_15260\1267129062.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appliances_dataset[appliance][appliance] = scaler.fit_transform(appliances_dataset[appliance][[appliance]])


Coffee Machine: min = 0.0, max = 1459.221
Target appliance 'Coffee Machine': min = 0.0, max = 1.0
Dryer: min = 0.0, max = 855.2519
Target appliance 'Dryer': min = 0.0, max = 1.0
Freezer: min = 0.0, max = 901.0220000000002
Target appliance 'Freezer': min = 0.0, max = 1.0
Fridge: min = 0.0, max = 1301.14
Target appliance 'Fridge': min = 0.0, max = 1.0
Lamp: min = 0.0, max = 130.521
Target appliance 'Lamp': min = 0.0, max = 1.0
Laptop: min = 0.0, max = 71.11036
Target appliance 'Laptop': min = 0.0, max = 0.9999999999999999
Microwave: min = 2.23214, max = 2637.667
Target appliance 'Microwave': min = 0.0, max = 1.0
PC: min = 25.195990000000002, max = 174.45499999999998
Target appliance 'PC': min = 0.0, max = 0.9999999999999999
Router: min = 18.91224, max = 21.471
Target appliance 'Router': min = 0.0, max = 0.9999999999999991
Tablet: min = 0.0, max = 9.006833
Target appliance 'Tablet': min = 0.0, max = 1.0
Washing Machine: min = 0.0, max = 2356.951
Target appliance 'Washing Machine': min = 0

In [8]:
def create_dataloader_for_appliance(dataset, input_columns, target_appliance, sequence_length, stride, batch_size, seed):
    sw_dataset = SlidingWindowDataset(dataset, input_columns, target_appliance, sequence_length=sequence_length, stride=stride)

    print('Dataset length:', len(sw_dataset), 'windows\n')

    total_len = len(sw_dataset)
    train_len = int(0.8 * total_len)
    val_len = int(0.1 * total_len)
    test_len = total_len - train_len - val_len

    # Split the dataset randomly
    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset, test_dataset = random_split(
        sw_dataset, [train_len, val_len, test_len], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, generator=generator)  # shuffle=True for training
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, generator=generator)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, generator=generator)

    # Get one sample to determine input and output dimensions
    sample_X, sample_y = train_dataset[0]
    seq_len = sample_X.shape[0]
    input_size = sample_X.shape[1]  # Number of input features
    output_size = sample_y.shape[1]  # Number of target appliances

    return train_loader, val_loader, test_loader, seq_len, input_size, output_size


In [9]:
def evaluate_model(model, test_loader, target_scalers, appliance_name, device='cpu', plot=True):
    import plotly.graph_objs as go

    model.eval()
    model.to(device)

    predictions_all = []
    y_test_all = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            batch_predictions = model(batch_X)

            predictions_all.append(batch_predictions.cpu())
            y_test_all.append(batch_y.cpu())

    predictions_tensor = torch.cat(predictions_all, dim=0)
    y_test_tensor = torch.cat(y_test_all, dim=0)

    pred_np = predictions_tensor.reshape(-1).numpy()
    y_test_np = y_test_tensor.reshape(-1).numpy()

    scaler = target_scalers[appliance_name]
    predictions_denorm = scaler.inverse_transform(pred_np.reshape(-1, 1)).flatten()
    y_test_denorm = scaler.inverse_transform(y_test_np.reshape(-1, 1)).flatten()

    mae = np.mean(np.abs(predictions_denorm - y_test_denorm))
    mse = np.mean((predictions_denorm - y_test_denorm) ** 2)
    rmse = np.sqrt(mse)
    sae = np.abs(np.sum(predictions_denorm) - np.sum(y_test_denorm)) / (np.sum(y_test_denorm) + 1e-8)
    nde = np.linalg.norm(predictions_denorm - y_test_denorm) / (np.linalg.norm(y_test_denorm) + 1e-8)

    print(f"Validation MAE: {mae:.6f} W")
    print(f"Validation MSE: {mse:.6f} W²")
    print(f"Validation RMSE: {rmse:.6f} W")
    print(f"Signal Aggregate Error (SAE): {sae:.6f}")
    print(f"Normalized Disaggregation Error (NDE): {nde:.6f}")

    fig = None
    if plot:
        trace_actual = go.Scatter(
            y=y_test_denorm,
            mode='lines',
            name='Actual',
            line=dict(color='blue')
        )
        trace_predicted = go.Scatter(
            y=predictions_denorm,
            mode='lines',
            name='Predicted',
            line=dict(color='red')
        )
        layout = go.Layout(
            title=f'{appliance_name} - Power Consumption Prediction',
            xaxis=dict(title='Sample'),
            yaxis=dict(title='Power (W)'),
            width=900,
            height=400,
        )
        fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "SAE": sae,
        "NDE": nde,
        "predictions": predictions_denorm,
        "targets": y_test_denorm,
        "fig": fig
    }


### Model Training

In [10]:
def train_nilm_model(train_loader, val_loader, input_size, output_size, seq_len, device, hidden_size, max_epochs, patience, model_file, data_path, num_layers, seed):
    model = NILMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers).to(device)
    print('\n', model)

    criterion = torch.nn.SmoothL1Loss()
    optimizer = Adam(model.parameters(), lr=0.0001)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.9)

    # Initialize variables for tracking
    best_loss = float('inf')
    patience_counter = 0

    # To track loss
    train_losses = []
    val_losses = []

    # Training loop with early stopping
    for epoch in range(max_epochs):
        model.train()
        train_loss = 0.0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_X)

            loss = criterion(outputs, batch_y)
            loss.backward()

            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)

        print(f"Epoch [{epoch + 1}/{max_epochs}], Train Loss: {train_loss:.6f}")

        # Step the learning rate scheduler
        scheduler.step()

        # Evaluate on val set
        model.eval()
        with torch.no_grad():
            val_loss = 0.0

            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                outputs = model(batch_X)

                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

            # Average across batches
            val_loss = val_loss / len(val_loader)
            val_losses.append(val_loss)  # optional: save per appliance losses
            print('Validation Loss: {:.6f}'.format(val_loss))

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            model_cpu = model.cpu()
            example_input = torch.randn(1, seq_len, input_size)
            traced_model = torch.jit.trace(model_cpu, example_input)
            traced_model.save(os.path.join(data_path, models_dir, model_file))

            # Move model back to GPU for further training
            model = model.to(device)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

    # Plot training and validation loss
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(train_losses, label='Train Loss')
    ax.plot(val_losses, label='Validation Loss')
    ax.set_title("Training and Validation Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    plt.close(fig)  # Prevent the figure from displaying now

    return model, train_losses, val_losses, fig

# Training

In [11]:
def run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_len, stride, num_layers, save_dir='output_logs'):
    buffer = io.StringIO()
    os.makedirs(save_dir, exist_ok=True)

    html_combined_path = os.path.join(save_dir, f"{appliance_name.lower().replace(' ', '_')}_logs.html")

    with redirect_stdout(buffer):
        print(f"Training and evaluating model for: {appliance_name}")

        # Load dataset
        appliance_df = appliances_dataset[appliance_name]
        train_loader, val_loader, test_loader, _, input_size, output_size = create_dataloader_for_appliance(
            dataset=appliance_df,
            input_columns=input_columns,
            target_appliance=[appliance_name],
            sequence_length=sequence_length,
            stride=stride,
            batch_size=batch_size,
            seed=SEED
        )

        # Train and save model
        model_full_file = appliance_name.lower().replace(' ', '_') + model_file
        model, train_losses, val_losses, loss_fig = train_nilm_model(
            train_loader=train_loader,
            val_loader=val_loader,
            input_size=input_size,
            output_size=output_size,
            seq_len=seq_len,
            device=device,
            hidden_size=hidden_size,
            max_epochs=max_epochs,
            patience=patience,
            model_file=model_full_file,
            data_path=data_path,
            num_layers=num_layers,
            seed=SEED,
        )

        buf = io.BytesIO()
        loss_fig.savefig(buf, format='png')
        plt.close(loss_fig)  # close the figure to free memory
        buf.seek(0)
        # Encode to base64 string
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')

        html_img_tag = f'<img src="data:image/png;base64,{img_base64}" alt="Training Loss Plot" />'


        # Evaluate model
        print(f"\nEvaluating model for: {appliance_name}")
        model = torch.jit.load(os.path.join(data_path, models_dir, model_full_file), map_location='cpu')
        results = evaluate_model(model, test_loader, target_scalers, appliance_name, device='cpu', plot=True)

    # Grab text output
    output_text = buffer.getvalue()

    # Get Plotly figure from results dict, if available
    fig = results.get("fig")

    # Prepare the full HTML content combining logs + plotly figure HTML
    fig_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    combined_html = f"""
    <html>
    <head><title>{appliance_name} Logs and Plots</title></head>
    <body>
      <h2>Console Output</h2>
      <pre style="background:#f0f0f0; padding:10px; border:1px solid #ccc; white-space: pre-wrap;">
{output_text}
      </pre>
      <hr>
      <h2>Training and Validation Loss</h2>
      {html_img_tag}
      <hr>
      <h2>Interactive Plot</h2>
      {fig_html}
    </body>
    </html>
    """

    # Write the combined HTML file
    with open(html_combined_path, 'w', encoding='utf-8') as f:
        f.write(combined_html)


## Coffee Machine

In [16]:
appliance_name = 'Coffee Machine'

hidden_size = 256
seq_length = 120
stride = 0.25
num_layers = 3

# POTENZIALMENTE DA RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Dryer

In [12]:
appliance_name = 'Dryer'

hidden_size = 256
seq_length = 120
stride = 0.1
num_layers = 5

# ok

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Freezer

In [17]:
appliance_name = 'Freezer'

hidden_size = 256
seq_length = 120
stride = 0.1
num_layers = 5

# RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Fridge

In [14]:
appliance_name = 'Fridge'

hidden_size = 128
seq_length = 240
stride = 0.25
num_layers = 3

# RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Lamp

In [14]:
appliance_name = 'Lamp'

hidden_size = 256
seq_length = 120
stride = 0.1
num_layers = 5

# POTENZIALMENTE DA RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

~~~~## Laptop

In [12]:
appliance_name = 'Laptop'

hidden_size = 256
seq_length = 120
stride = 0.25
num_layers = 4

# POTENZIALMENTE DA RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Microwave

In [13]:
appliance_name = 'Microwave'

hidden_size = 256
seq_length = 120
stride = 0.1
num_layers = 4

# RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## PC

In [14]:
appliance_name = 'PC'

hidden_size = 128
seq_length = 360
stride = 0.25
num_layers = 3

# RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Router

In [13]:
appliance_name = 'Router'

hidden_size = 256
seq_length = 120
stride = 0.25
num_layers = 3

# Ok

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Tablet

In [15]:
appliance_name = 'Tablet'

hidden_size = 128
seq_length = 120
stride = 0.25
num_layers = 4

# PRunnare così

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Washing Machine

In [19]:
appliance_name = 'Washing Machine'

hidden_size = 256
seq_length = 120
stride = 0.1
num_layers = 4

# RIFARE

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)

## Other

In [18]:
appliance_name = 'Other'

hidden_size = 256
seq_length = 360
stride = 0.25
num_layers = 4

# Ok

run_training_and_evaluation(appliance_name, appliances_dataset, hidden_size, seq_length, int(seq_length * stride), num_layers)